<a href="https://colab.research.google.com/github/alexandreib/QuantDesign/blob/main/SP500_Tickers_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Portfolio Optimization on S&P 500 Stocks**




Imports :


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt

# Download the Stocks Returns


## Scraping SP500 Tickers

In [2]:
# Download the S&P 500 constituents from Wikipedia
try:
    table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    df = table[0]
    tickers = df['Symbol'].tolist()
except Exception as e:
    print(f"Error downloading S&P 500 tickers: {e}")
    tickers = [] # Handle the error gracefully, e.g., provide a default list

# Print or use the tickers list
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMTM', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', '

## Download the last 20 years of price

In [3]:
# Define the start and end dates for the data
end_date = pd.Timestamp.today()
start_date = end_date - pd.Timedelta(days=365 * 10)

# Download the data
df = yf.download(tickers, start=start_date, end=end_date)

# Print the data (optional)
df.head()

[*********************100%***********************]  503 of 503 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2014-11-07 12:48:39.210812 -> 2024-11-04 12:48:39.210812)')
ERROR:yfinance:['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Price                      Adj Close                                        \
Ticker                             A       AAPL       ABBV ABNB        ABT   
Date                                                                         
2014-11-07 00:00:00+00:00  37.731159  24.302616  40.660728  NaN  35.844234   
2014-11-10 00:00:00+00:00  38.284256  24.262487  42.174759  NaN  36.439297   
2014-11-11 00:00:00+00:00  38.404099  24.456446  42.214420  NaN  36.571529   
2014-11-12 00:00:00+00:00  38.210514  24.802000  42.154926  NaN  36.555008   
2014-11-13 00:00:00+00:00  38.210514  25.152012  42.068985  NaN  36.852535   

Price                                                                  \
Ticker                          ACGL        ACN       ADBE        ADI   
Date                                                                    
2014-11-07 00:00:00+00:00  19.160000  70.768875  71.599998  40.681759   
2014-11-10 00:00:00+00:00  19.086666  71.345329  71.699997  40.827518   
2014-11-11 00:00:00+00:00  19.110001  71.158829  71.699997  40.600780   
2014-11-12 00:00:00+00:00  19.139999  71.311455  71.410004  40.697956   
2014-11-13 00:00:00+00:00  19.200001  71.582703  71.720001  40.657475   

Price                                 ...  Volume                             \
Ticker                           ADM  ...     WTW       WY     WYNN      XEL   
Date                                  ...                                      
2014-11-07 00:00:00+00:00  38.208622  ...  369837  2184200  1489600  4070000   
2014-11-10 00:00:00+00:00  38.374901  ...  309814  2233600  1034000  3284100   
2014-11-11 00:00:00+00:00  38.639450  ...  291505  2212500  1296400  2469700   
2014-11-12 00:00:00+00:00  38.329556  ...  324537  4874700  2121700  3203100   
2014-11-13 00:00:00+00:00  38.405136  ...  240769  1644200  1294400  2690900   

Price                                                                  \
Ticker                          XOM      XYL      YUM     ZBH    ZBRA   
Date                                                                    
2014-11-07 00:00:00+00:00  13233500   669300  3368446  854694  507200   
2014-11-10 00:00:00+00:00   9087600   625800  3756396  826163  318300   
2014-11-11 00:00:00+00:00   7990000   534600  2132959  585555  293800   
2014-11-12 00:00:00+00:00  11592800   762500  3970331  806490  333500   
2014-11-13 00:00:00+00:00  12355400  1109100  5418501  836154  496700   

Price                                
Ticker                          ZTS  
Date                                 
2014-11-07 00:00:00+00:00   6819000  
2014-11-10 00:00:00+00:00   7530100  
2014-11-11 00:00:00+00:00  20228500  
2014-11-12 00:00:00+00:00  21414900  
2014-11-13 00:00:00+00:00   6151300  

[5 rows x 3018 columns]

## Reshape, and Clean the DataFrame

In [4]:
# Reshape the DataFrame
df = df['Adj Close'].reset_index()
df = pd.melt(df, id_vars='Date', value_vars=tickers, var_name='Ticker', value_name='Adj Close')
df['Date'] = pd.to_datetime(df['Date']).dt.date
df = df.rename(columns={'index': 'Ticker'})

# Print or use the transformed DataFrame
df.head()

,Date,Ticker,Adj Close
0,2014-11-07,MMM,93.436005
1,2014-11-10,MMM,94.098495
2,2014-11-11,MMM,93.770256
3,2014-11-12,MMM,94.110443
4,2014-11-13,MMM,94.510338


# Prepare the Returns

## Log Returns

Using log returns instead of simple returns offers several advantages:

- **Time Additivity:**  
  Log returns can be summed over different periods for cumulative return calculations, unlike simple returns.

- **Compounding:**  
  Log returns naturally account for compounding, providing a more accurate measure of investment growth.

- **Normality Assumption:**  
  Log returns are often more normally distributed, aiding statistical modeling and risk management.

- **Symmetry:**  
  Log returns treat gains and losses symmetrically, better reflecting the continuous nature of returns.


In [9]:
# Calculate daily log returns
df['Log_Return'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))

# # Calculate quarterly log returns Using the first advantage of log Rturns : Tie Additivity
df['Quarterly_Log_Return'] = df.groupby('Ticker')['Daily_Log_Return'].rolling(window=63).sum().reset_index(0,drop=True)

# # Creation of the log return Matrix
# log_returns_matrix = pd.pivot_table(df[['Date','Ticker', 'Quarterly_Log_Return']], index ='Date', columns = 'Ticker', aggfunc='mean')
# log_returns_matrix = log_returns_matrix.droplevel(0, axis = 'columns')
# log_returns_matrix.head(1)


In [10]:
df

,Date,Ticker,Adj Close,Daily_Log_Return,Quarterly_Log_Return,Log_Return
0,2014-11-07,MMM,93.436005,NaN,NaN,NaN
1,2014-11-10,MMM,94.098495,0.007065,NaN,0.007065
2,2014-11-11,MMM,93.770256,-0.003494,NaN,-0.003494
3,2014-11-12,MMM,94.110443,0.003621,NaN,0.003621
4,2014-11-13,MMM,94.510338,0.004240,NaN,0.004240
...,...,...,...,...,...,...
1264034,2024-10-28,ZTS,182.327942,0.015161,0.005102,0.015161
1264035,2024-10-29,ZTS,180.841476,-0.008186,0.006809,-0.008186
1264036,2024-10-30,ZTS,182.307999,0.008077,0.001753,0.008077
1264037,2024-10-31,ZTS,178.779999,-0.019542,-0.007706,-0.019542
